# Docking and Scoring
## Intro

This notebook will show you how to dock and score molecules using the asapdiscovery-docking module. You should have already gone through the `interfacing with databases and systems` tutorial. 


Our docking pipeline primarily focuses on the use-case for a structure-enabled drug discovery program, in which we have crystal structures of early molecules to use for *reference-based* docking. This approach has been demonstrated to be effective for prioritising designed compounds. 

To this end, we have implemented an api that wraps the OpenEye POSIT docking algorithm, which, through its use of the HYBRID and SHAPEFIT algorithms, enables reference-based docking. 

### The scope of this guide

This guide will show you how to dock and score molecules. For the *extremely* necessary precursor step of data loading and prepping, please see [protein_and_ligand_prep](%protein_and_ligand_prep.ipynb)

## Setting up example data

We will use files we use for testing, since these molecules have already been prepped for docking.

In [1]:
from asapdiscovery.data.testing.test_resources import fetch_test_file
from asapdiscovery.data.schema.complex import Complex, PreppedComplex
from asapdiscovery.data.schema.ligand import Ligand

# grab a pre-prepared complex 
prepped_complex = PreppedComplex.from_oedu_file(
        fetch_test_file("Mpro-P2660_0A_bound-prepped_receptor.oedu"),
        ligand_kwargs={"compound_name": "test"},
        target_kwargs={"target_name": "test", "target_hash": "mock_hash"},
    )

# make a ligand from an SDF file 
ligand = Ligand.from_sdf(
        fetch_test_file("Mpro-P0008_0A_ERI-UCB-ce40166b-17.sdf"), compound_name="test"
    )

## Docking with the POSITDocker

There are a *ton* of choices we can make for docking, which will not be enumerated here. But in order to get a flavor for the options, we can examine the class attributes of the POSITDocker:

In [2]:
from asapdiscovery.docking.openeye import POSITDocker

In [3]:
POSITDocker?

Init signature:
POSITDocker(
    *,
    type: Literal['POSITDocker'] = 'POSITDocker',
    relax: asapdiscovery.docking.openeye.POSIT_RELAX_MODE = <POSIT_RELAX_MODE.NONE: 0>,
    posit_method: asapdiscovery.docking.openeye.POSIT_METHOD = <POSIT_METHOD.ALL: 15>,
    use_omega: bool = True,
    omega_dense: bool = False,
    num_poses: pydantic.types.PositiveInt = 1,
    allow_low_posit_prob: bool = False,
    low_posit_prob_thresh: float = 0.1,
    allow_final_clash: bool = False,
    allow_retries: bool = True,
) -> None
Docstring:      Base class for running docking
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /store/projects/ASAP_dev/asapdiscovery/asapdiscovery-docking/asapdiscovery/docking/openeye.py
Type:           ModelMetaclass
Subclasses:     

A quick overview of some important options 

* `relax`: Whether to allow relaxation when generating docked structures
* `posit_method`: which POSIT method to use, see [here](https://docs.eyesopen.com/applications/oedocking/theory/posit_theory.html#) for a complete treatment of POSIT theory. The default (`ALL`) selects the best method possible iteratively.
* `use_omega`: Use OpenEye's OMEGA conformer generation to enumerate conformers before docking. Should vastly improve the quality of predicted poses.
* `omega_dense`: Use dense OMEGA sampling.
* `allow_retries`: Try several configurations of docking parameters to attempt to obtain a result. 

In [4]:
# lets go ahead and make a Docker object
docker = POSITDocker()

In [5]:
# and check out its defaults.
docker.dict()

{'type': 'POSITDocker',
 'relax': <POSIT_RELAX_MODE.NONE: 0>,
 'posit_method': <POSIT_METHOD.ALL: 15>,
 'use_omega': True,
 'omega_dense': False,
 'num_poses': 1,
 'allow_low_posit_prob': False,
 'low_posit_prob_thresh': 0.1,
 'allow_final_clash': False,
 'allow_retries': True}

`POSITDocker.dock()` requires:
1) a list of DockingInputBase objects
2) an output directory
3) and some dask options

Currently, we have 2 kinds of DockingInputBase objects implemented:
1) a complex-ligand pair (DockingInputPair)
2) a one-to-many ligand:complexes object (DockingInputMultiStructure)

### Running simple docking 

In [6]:
from asapdiscovery.docking.docking import DockingInputPair
input_pair = DockingInputPair(ligand=ligand, complex=prepped_complex)

In [7]:
results = docker.dock(inputs=[input_pair])

This returns a list of POSITDockingResults objects!

In [8]:
# lets grab one of the results objects
result = results[0] 

In [9]:
# and dump it to disk
result.write_docking_files("docking_test")

## Scoring

We decouple the *pose prediction* and *scoring* parts of docking, which enables us to score docked poses with different scoring functions. 

This flexibility allows us to implement our own scoring functions and capture information important to our discovery process. 

To this end, we have written a few "scorer" classes, including:
1. A traditional phyiscs based docking scorer: `ChemGauss4Scorer`
2. A score which tries to capture information about the potential for the binding site to evolve: `FINTScorer` (see `working_with_fitness_data` tutorial)
3. A 2D Graph Attention based scorer trained on Covid Moonshot data that predicts pIC50s directly: `GATScorer`
4. 3D equivariant scorers that use 3D pose information and are trained on Covid Moonshot data to predict pIC50s:  `E3NNScorer` and  `SchnetScorer`
5. And finally, a MetaScorer which can run all the other scorers easily. 

In [10]:
from asapdiscovery.docking.scorer import ChemGauss4Scorer, FINTScorer, MetaScorer

### Targets

Several of our scorers require target-specific information. We can find out the targets that the repo "knows about" like so:

In [11]:
from asapdiscovery.data.services.postera.manifold_data_validation import TargetTags

In [12]:
TargetTags.get_values()

['SARS-CoV-2-Mpro',
 'EV-A71-Capsid',
 'EV-D68-Capsid',
 'ZIKV-NS2B-NS3pro',
 'DENV-NS2B-NS3pro',
 'SARS-CoV-2-N-protein',
 'EV-D68-3Cpro',
 'MERS-CoV-Mpro',
 'SARS-CoV-2-Mac1',
 'EV-A71-3Cpro']

Since we're working with a known target, we can set that as a variable and use that throughout

In [13]:
target = TargetTags("SARS-CoV-2-Mpro")

### ChemGauss4 Scorer

In [14]:
chemgauss_scorer = ChemGauss4Scorer()

In [15]:
scores = chemgauss_scorer.score(results)
scores

[Score(score_type=<ScoreType.chemgauss4: 'chemgauss4'>, score=-11.036272048950195, compound_name='test', smiles='c1ccc2c(c1)c(cc(=O)[nH]2)C(=O)NCCOc3cc(cc(c3)Cl)O[C@H]4CC(=O)N4', ligand_identifiers=None, ligand_inchikey='SNQHLYWSRJDLGK-FQEVSTJZSA-N', target_name='test', target_identifiers=None, complex_ligand_smiles='CNC(=O)CN1C[C@]2(CCN(C2=O)c3cncc4c3cc(cc4)Cl)c5cc(ccc5C1=O)Cl', probability=0.18000000715255737, units=<ScoreUnits.arbitrary: 'arbitrary'>)]

We can see this returns an array of score objects. If we want a dataframe, we can ask it to run this instead:

In [16]:
scores_df = chemgauss_scorer.score(results, return_df=True)
scores_df

,score_type,score,compound_name,smiles,ligand_identifiers,ligand_inchikey,target_name,target_identifiers,complex_ligand_smiles,probability,units
0,chemgauss4,-11.036272,test,c1ccc2c(c1)c(cc(=O)[nH]2)C(=O)NCCOc3cc(cc(c3)C...,None,SNQHLYWSRJDLGK-FQEVSTJZSA-N,test,None,CNC(=O)CN1C[C@]2(CCN(C2=O)c3cncc4c3cc(cc4)Cl)c...,0.18,ScoreUnits.arbitrary


### FINTScore

In antiviral drug discovery, potential for mutation of target binding sites is very high. Thus it is important to avoid sidechain-ligand interactions with highly mutable residues when evaluating potential designs. 

FINTScore attempts to compress information about the mutability of the ligand binding site into a single number between 0 and 1. It rewards interactions with non-mutable sidechains and backbone atoms, while penalising interactions with mutable sidechains. See the implementation for more details. You can also view this information in 3D. See the `visualising ASAP targets` tutorial for more information. 

For the FINT score, we need fitness data (normally obtained by deep mutational scanning experiments or from phylogenetic data), which means we can only work on a target for which we have vendored fitness data. To check which targets those are, we can use:

In [17]:
from asapdiscovery.genetics.fitness import target_has_fitness_data

# does our target have fitness data?
target_has_fitness_data(target) # yes!

True

In [18]:
fint_scorer = FINTScorer(target=target)

In [19]:
scores = fint_scorer.score(results, return_df=True)
scores

2024-05-06 10:55:36,616 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-05-06 10:55:36,616 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-05-06 10:55:36,616 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-05-06 10:55:36,616 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of tmp_complex.pdb
2024-05-06 10:55:36,752 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in /tmp/tmpurafp1ze/


,score_type,score,compound_name,smiles,ligand_identifiers,ligand_inchikey,target_name,target_identifiers,complex_ligand_smiles,probability,units
0,FINT,0.200455,test,c1ccc2c(c1)c(cc(=O)[nH]2)C(=O)NCCOc3cc(cc(c3)C...,None,SNQHLYWSRJDLGK-FQEVSTJZSA-N,test,None,CNC(=O)CN1C[C@]2(CCN(C2=O)c3cncc4c3cc(cc4)Cl)c...,0.18,ScoreUnits.arbitrary


### ML Scorers

Our ML scorers are trained to predict pIC50s from 2D graph or 3D equivariant representations of ligands and target-ligand complexes respectively. This is enabled by the [MTENN](https://github.com/choderalab/mtenn) framework that abstracts and modularize the task of structure-based machine learning.

Currently we have ML scorers for some targets seperately, but are exploring PDBBind based foundational models for multi-target prediction (watch this space).  

In [20]:
# import our ML model registry
from asapdiscovery.ml.models import ASAPMLModelRegistry

In [21]:
ASAPMLModelRegistry.get_implemented_model_types()

['GAT', 'e3nn', 'schnet']

In [22]:
from asapdiscovery.docking.scorer import MLModelScorer
ml_scorers = [MLModelScorer.from_latest_by_target_and_type(target, model_type) 
           for model_type in ASAPMLModelRegistry.get_implemented_model_types()]

/home/hmacdope/mambaforge/envs/asapdiscovery6/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/hmacdope/mambaforge/envs/asapdiscovery6/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/hmacdope/mambaforge/envs/asapdiscovery6/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/hmacdope/mambaforge/envs/asapdiscovery6/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/home/hmacdope/mambaforge/envs/asapdiscovery6/lib/python3.10/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on

In [23]:
gat_scores = ml_scorers[0].score(results, return_df=True)

In [24]:
gat_scores

,score_type,score,compound_name,smiles,ligand_identifiers,ligand_inchikey,target_name,target_identifiers,complex_ligand_smiles,probability,units
0,GAT,4.912028,test,c1ccc2c(c1)c(cc(=O)[nH]2)C(=O)NCCOc3cc(cc(c3)C...,None,SNQHLYWSRJDLGK-FQEVSTJZSA-N,test,None,CNC(=O)CN1C[C@]2(CCN(C2=O)c3cncc4c3cc(cc4)Cl)c...,0.18,ScoreUnits.pIC50


#### MetaScorer

We can use the MetaScorer to run all the scoring for us and combine everything into a dataframe we can save easily

In [25]:
scorers = [chemgauss_scorer, fint_scorer, *ml_scorers]

In [26]:
metascorer = MetaScorer(scorers=scorers)

In [27]:
scores_df = metascorer.score(results, return_df=True)

2024-05-06 10:55:42,241 [INFO] [plipcmd.py:124] plip.plipcmd: Protein-Ligand Interaction Profiler (PLIP) 2.3.0
2024-05-06 10:55:42,241 [INFO] [plipcmd.py:125] plip.plipcmd: brought to you by: PharmAI GmbH (2020-2021) - www.pharm.ai - hello@pharm.ai
2024-05-06 10:55:42,241 [INFO] [plipcmd.py:126] plip.plipcmd: please cite: Adasme,M. et al. PLIP 2021: expanding the scope of the protein-ligand interaction profiler to DNA and RNA. Nucl. Acids Res. (05 May 2021), gkab294. doi: 10.1093/nar/gkab294
2024-05-06 10:55:42,241 [INFO] [plipcmd.py:49] plip.plipcmd: starting analysis of tmp_complex.pdb
2024-05-06 10:55:42,381 [INFO] [plipcmd.py:165] plip.plipcmd: finished analysis, find the result files in /tmp/tmpon9zszjf/


In [28]:
scores_df

score_type,target_identifiers,ligand_id,docking-confidence-POSIT,complex_ligand_smiles,INCHIKEY,ligand_identifiers,docking-structure-POSIT,SMILES,fitness-score-FINT,computed-GAT-pIC50,docking-score-POSIT,computed-E3NN-pIC50,computed-SchNet-pIC50
0,NaN,test,0.18,CNC(=O)CN1C[C@]2(CCN(C2=O)c3cncc4c3cc(cc4)Cl)c...,SNQHLYWSRJDLGK-FQEVSTJZSA-N,NaN,test,c1ccc2c(c1)c(cc(=O)[nH]2)C(=O)NCCOc3cc(cc(c3)C...,0.200455,4.912028,-11.036272,5.281067,-0.023069


Under the hood, this uses this function: `asapdiscovery.docking.scorer.Score._combine_and_pivot_scores_df` to return the scores in a dataframe. As of version 0.4, this uses `asapdiscovery.docking.scorer._SCORE_MANIFOLD_ALIAS` to change the column names to conform to the standard names used within the ASAPDiscovery Consortium. You can examine which column names correspond to which score here:

In [29]:
from asapdiscovery.docking.scorer import _SCORE_MANIFOLD_ALIAS
_SCORE_MANIFOLD_ALIAS

{<ScoreType.chemgauss4: 'chemgauss4'>: 'docking-score-POSIT',
 <ScoreType.FINT: 'FINT'>: 'fitness-score-FINT',
 <ScoreType.GAT: 'GAT'>: 'computed-GAT-pIC50',
 <ScoreType.schnet: 'schnet'>: 'computed-SchNet-pIC50',
 <ScoreType.e3nn: 'e3nn'>: 'computed-E3NN-pIC50',
 <ScoreType.INVALID: 'INVALID'>: None,
 'target_name': 'docking-structure-POSIT',
 'compound_name': 'ligand_id',
 'smiles': 'SMILES',
 'ligand_inchikey': 'INCHIKEY',
 'probability': 'docking-confidence-POSIT'}

## Advanced Topics: Selectors

The ASAPDiscovery Consortium regularly operates in a regime where we have many experimental structures to choose from as references for docking. To accelerate the process of choosing which structures to use, we have generated a series of Selector objects which take a set of ligands and complexes and choose which set of ligand-complex pairs to use for docking.

In [30]:
from asapdiscovery.data.operators.selectors.selector_list import StructureSelector

In [31]:
StructureSelector.get_values()

['MCSSelector',
 'PairwiseSelector',
 'LeaveOneOutSelector',
 'LeaveSimilarOutSelector',
 'SelfDockingSelector']

In [32]:
# mock a fragalysis dump from diamond light source

from asapdiscovery.data.services.fragalysis.fragalysis_reader import FragalysisFactory
all_mpro_fns = [
        "metadata.csv",
        "aligned/Mpro-x11041_0A/Mpro-x11041_0A_bound.pdb",
        "aligned/Mpro-x1425_0A/Mpro-x1425_0A_bound.pdb",
        "aligned/Mpro-x11894_0A/Mpro-x11894_0A_bound.pdb",
        "aligned/Mpro-x1002_0A/Mpro-x1002_0A_bound.pdb",
        "aligned/Mpro-x10155_0A/Mpro-x10155_0A_bound.pdb",
        "aligned/Mpro-x0354_0A/Mpro-x0354_0A_bound.pdb",
        "aligned/Mpro-x11271_0A/Mpro-x11271_0A_bound.pdb",
        "aligned/Mpro-x1101_1A/Mpro-x1101_1A_bound.pdb",
        "aligned/Mpro-x1187_0A/Mpro-x1187_0A_bound.pdb",
        "aligned/Mpro-x10338_0A/Mpro-x10338_0A_bound.pdb",
    ]
all_paths = [fetch_test_file(f"frag_factory_test/{fn}") for fn in all_mpro_fns]
parent_dir = all_paths[0].parent



In [33]:
ff = FragalysisFactory.from_dir(parent_dir)
complexes = ff.load()

In [34]:
ligands = [complex.ligand for complex in complexes]

### To illustrate what the selectors do

Selectors take a list of `Complexes` and `Ligands` and return `Pairs` based on some criteria. 

For example the `SelfDockingSelector` only selects ligands that match those already present in the complexes. 

#### SelfDockingSelector

In [35]:
selector = StructureSelector('SelfDockingSelector').selector_cls()

In [36]:
pairs = selector.select(ligands, complexes)

In [37]:
len(pairs)

10

In [38]:
# are all the ligands the same as those in the complexes?
all(pair.complex.ligand == pair.ligand for pair in pairs)

True

#### PairwiseSelector

The `PairwiseSelector` produces the full outer product of complexes and ligands e.g in the example below we have 10 complexes and 10 ligands. The resulting outer product of pairs contains 100 elements. 

In [66]:
selector = StructureSelector('PairwiseSelector').selector_cls()

In [67]:
pairs = selector.select(ligands, complexes)

In [68]:
len(pairs)

100

#### MCSSelector

The `MCSSelector` selects complexes that closely match the structures in the query ligand by Maximum Common Substructure (MCS). This is very useful when docking new designs and you need to determine a chemically similar reference. 

In [69]:
selector = StructureSelector('MCSSelector').selector_cls()

In [73]:
# the n_select parameter controls how many matches per ligand
pairs = selector.select(ligands, complexes, n_select=5)

TypeError: LeaveSimilarOutSelector._select() got an unexpected keyword argument 'n_select'

In [71]:
len(pairs)

50

#### LeaveSimilarOutSelector

The `LeaveSimilarOutSelector` filters out pairs where the ligand is a stereoisomer / tautomer / protonation state isomer / etc of the complex ligand. It can take a while though because it has to do an len(ligands) * len(complexes) pairwise comparison of all those chemical possibilities. 

In [74]:
selector = StructureSelector('LeaveSimilarOutSelector').selector_cls()

In [75]:
pairs = selector.select(ligands, complexes)

In [47]:
len(pairs)

90

## Advanced Topics: Multi-Structure Docking


Some docking protocols (i.e., POSIT) will accept multiple receptor structures and choose for themselves which to dock to. For these docking protocols, we pass a different kind of input:

In [48]:
from asapdiscovery.docking.docking import DockingInputMultiStructure
from asapdiscovery.data.operators.selectors.selector_list import StructureSelector

In [49]:
cached_dus = {
        "Mpro-x1002": "du_cache/Mpro-x1002_0A_bound.oedu",
        "Mpro-x0354": "du_cache/Mpro-x0354_0A_bound.oedu",
    }
prepped_complexes = [
        PreppedComplex.from_oedu_file(
            fetch_test_file(cached_du),
            ligand_kwargs={"compound_name": "test"},
            target_kwargs={"target_name": name, "target_hash": "mock_hash"},
        )
        for name, cached_du in cached_dus.items()
    ]
ligand = Ligand.from_sdf(
        fetch_test_file("Mpro-P0008_0A_ERI-UCB-ce40166b-17.sdf"), compound_name="test"
    )

Let's assume we had gotten this subset of ligand-protein pairs from the selector logic from above. This would look something like this:

In [50]:
selector = StructureSelector('LeaveSimilarOutSelector').selector_cls()
pairs = selector.select([ligand], prepped_complexes)
len(pairs)

2

We then collapse these pairs into a single MultiStructure set:

In [51]:
inputs = DockingInputMultiStructure.from_pairs(pairs) # Returns a list since multiple sets could be generated

If we already knew exactly what we wanted to do, we could just create the set directly:

In [52]:
alternate_inputs = DockingInputMultiStructure(ligand=ligand, complexes=prepped_complexes)

We can see that two are equivalent in this case:

In [53]:
inputs[0] == alternate_inputs

True

Now we run docking as before:

In [54]:
docker = POSITDocker() # let's just use defaults for now

In [55]:
results = docker.dock(inputs) # we won't use dask or write an output, takes ~3 minutes on a Macbook Pro

In [56]:
result = results[0]

In [57]:
result.write_docking_files("multi_structure_docking_test")

Since we input multiple structures, we don't know which one it actually used. We can find this out by examining the results:

In [58]:
result.input_pair.complex.target.target_name

'Mpro-x0354'

## Advanced Topics: Multi-pose Docking

Note: this is functionality that was most recently added. Please make an issue if you encounter problems :)

We'll use the same docking scheme as above

In [59]:
from asapdiscovery.docking.docking import DockingInputPair
prepped_complex = PreppedComplex.from_oedu_file(
        fetch_test_file("Mpro-P2660_0A_bound-prepped_receptor.oedu"),
        ligand_kwargs={"compound_name": "test"},
        target_kwargs={"target_name": "test", "target_hash": "mock_hash"},
    )
ligand = Ligand.from_sdf(
        fetch_test_file("Mpro-P0008_0A_ERI-UCB-ce40166b-17.sdf"), compound_name="test"
    )
input_pair = DockingInputPair(ligand=ligand, complex=prepped_complex)

In [60]:
docker = POSITDocker(num_poses=50) # we set the number of poses when we create the docker

In [61]:
results = docker.dock([input_pair]) # we won't use dask or write an output, takes ~1 min on a Macbook Pro

NOTE: As of version 0.3.1, we can generate multipose docking results, but there's nothing to distinguish them, and when you run result.write_docking_files the results for each ligand will overwrite each other. This should be fixed by 0.4.0

In [62]:
len(results)

29

In [63]:
print([result.probability for result in results])

[0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.23999999463558197, 0.18000000715255737, 0.18000000715255737, 0.18000000715255737, 0.18000000715255737, 0.18000000715255737, 0.18000000715255737, 0.18000000715255737, 0.05000000074505806, 0.05000000074505806, 0.05000000074505806, 0.05000000074505806]


## A few side notes: Dask and Target Specific Workflows

### Dask

We make heavy use of Dask throughout our code, which helps automate parallel processing and provides a nice dashboard for evaluating the progress of large scale docking efforts. Due to the way in which Dask automates error handling, this has occasionally led to situations where the behaviour of our code is different depending on whether you have enabled Dask. We have tried to stamp out any instances of this, but if you find another, please make an issue!

### Target-specific workflows

We have implemented our library code within the `asapdiscovery-workflows` module, which puts everything together in a command-line interface (cli). Unfortunately, as of version 0.4, these workflows only work if you are using the targets specified for ASAP. We plan on changing this for version 0.5 

To find out which targets can be passed to these workflows, you can use this:

In [64]:
from asapdiscovery.data.services.postera.manifold_data_validation import TargetTags

In [65]:
TargetTags.get_values()

['SARS-CoV-2-Mpro',
 'EV-A71-Capsid',
 'EV-D68-Capsid',
 'ZIKV-NS2B-NS3pro',
 'DENV-NS2B-NS3pro',
 'SARS-CoV-2-N-protein',
 'EV-D68-3Cpro',
 'MERS-CoV-Mpro',
 'SARS-CoV-2-Mac1',
 'EV-A71-3Cpro']